In [27]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Kumpulan Tugas/Rekdat Akhir'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Kumpulan Tugas/Rekdat Akhir


In [28]:
import xgboost as xgb
import pandas as pd

In [29]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv("election_summary.csv")
df = df.dropna()

In [31]:
df

,District,D Positive Tweet,D Negative Tweet,R Positive Tweet,R Negative Tweet,Raw PVI,2020 Biden %,2020 Trump %,2016 Clinton %,2016 Trump %,D Popularity,R Popularity,Winners
0,AK00,0.765766,0.234234,0.656522,0.343478,-8.02,0.428,0.528,0.366,0.513,105000,538880,D
1,AL01,0.000000,0.000000,0.612903,0.387097,-16.46,0.352,0.635,0.338,0.629,0,228000,R
2,AL04,0.677419,0.322581,0.852459,0.147541,-33.24,0.186,0.802,0.180,0.790,138000,7360,R
3,AL02,0.900000,0.100000,0.759615,0.240385,-17.32,0.347,0.641,0.323,0.648,8000,249000,R
4,AL03,0.800000,0.200000,0.902655,0.097345,-19.36,0.324,0.664,0.311,0.657,101,2490000,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,WI07,0.678571,0.321429,0.654867,0.345133,-12.27,0.392,0.591,0.369,0.572,709,3000000,R
431,WI08,0.000000,0.000000,0.735294,0.264706,-10.16,0.414,0.569,0.385,0.556,0,375000,R
432,WV01,0.831081,0.168919,0.734940,0.265060,-23.23,0.288,0.697,0.258,0.687,19100,280000,R
433,WV02,0.800000,0.200000,0.629921,0.370079,-21.52,0.306,0.676,0.265,0.670,25200,41600,R


In [37]:
X = df.drop(["District","Winners"],axis=1)
y= df["Winners"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [38]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier()

In [39]:
print(metrics.classification_report(y_test, model.predict(X_test)))
print(metrics.confusion_matrix(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           D       0.95      0.95      0.95        41
           R       0.96      0.96      0.96        46

    accuracy                           0.95        87
   macro avg       0.95      0.95      0.95        87
weighted avg       0.95      0.95      0.95        87

[[39  2]
 [ 2 44]]


In [41]:
pd.DataFrame.from_dict(model.get_booster().get_score(importance_type='gain'),orient='index').sort_values(0,ascending=False)

,0
Raw PVI,31.642874
2020 Biden %,22.089578
2016 Clinton %,10.947194
2020 Trump %,2.931718
R Positive Tweet,1.156180
2016 Trump %,0.923597
R Popularity,0.911287
R Negative Tweet,0.874178
D Popularity,0.833355
D Positive Tweet,0.559869


In [42]:
pd.DataFrame.from_dict(model.get_booster().get_score(importance_type='weight'),orient='index').sort_values(0,ascending=False)

,0
2020 Trump %,96
D Popularity,76
R Positive Tweet,67
R Popularity,65
2016 Clinton %,46
2020 Biden %,31
2016 Trump %,27
D Positive Tweet,25
Raw PVI,22
R Negative Tweet,5


In [43]:
model.save_model("us_election_xgb_model.json")